### Условие

Даны 2 таблицы. Первая содержит информацию о пополнениях
клиентами своих рублёвых или валютных счетов, во второй содержатся курсы
некоторых валют на дату.
Ответьте на следующие вопросы используя python или SQL (приложите код
решения):
1) Выведите максимальный размер пополнения в долларах за неделю,
начинающуюся 6-го февраля.
2) Выведите средний размер пополнений за февраль понедельно, в
разбивке по валютам.
Результатом должна быть таблица формата «Год-номер недели, валюта,
размер пополнения (с округлением до целого числа)».
3) Посчитайте долю клиентов, у которых были пополнения в нескольких
валютах. Ответ округлите до десятых процента.
4) По каждому клиенту вывести дату первого пополнения, валюту, сумму в
рублях (используйте вторую таблицу). Ответом выведите общую сумму в
рублях по всем клиентам.
5) Посчитайте по каждому клиенту среднюю разницу в днях между его
пополнениями, выведите медианное значение по всем клиентам.

### Решение

#### Подготовка данных

Импортируем необходимые библиотеки.

In [1]:
import pandas as pd

In [2]:
replenishment_of_accounts = pd.read_csv('replenishment_of_accounts.csv')
exchange_rates = pd.read_csv('exchange_rates.csv')

Рассмотрим датасет replenishment_of_accounts. Для обзора данных воспользуемся методами head() и info().

In [3]:
display(replenishment_of_accounts.head(3))
replenishment_of_accounts.info()

,operation_id,user_id,volume,currency,operation_date
0,6935122,29282488,25000.0,RUR,2023-02-07 11:17:28
1,5201349,82189896,600000.0,RUR,2023-02-06 14:39:08
2,6935148,9959035,51596.0,USD,2023-02-08 16:07:21


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26021 entries, 0 to 26020
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   operation_id    26021 non-null  int64  
 1   user_id         26021 non-null  int64  
 2   volume          26021 non-null  float64
 3   currency        26021 non-null  object 
 4   operation_date  26021 non-null  object 
dtypes: float64(1), int64(2), object(2)
memory usage: 1016.6+ KB


Пропущенных значений нет. 

В колонке operation_date можно изменить тип данных на datetime.

In [4]:
replenishment_of_accounts['operation_date'] = pd.to_datetime(replenishment_of_accounts['operation_date'], format = '%Y-%m-%d %H:%M:%S')
replenishment_of_accounts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26021 entries, 0 to 26020
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   operation_id    26021 non-null  int64         
 1   user_id         26021 non-null  int64         
 2   volume          26021 non-null  float64       
 3   currency        26021 non-null  object        
 4   operation_date  26021 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 1016.6+ KB


In [5]:
replenishment_of_accounts.head()

,operation_id,user_id,volume,currency,operation_date
0,6935122,29282488,25000.0,RUR,2023-02-07 11:17:28
1,5201349,82189896,600000.0,RUR,2023-02-06 14:39:08
2,6935148,9959035,51596.0,USD,2023-02-08 16:07:21
3,1733794,30118944,2.5,USD,2023-02-06 15:22:53
4,3467597,26949080,346.7,RUR,2023-02-09 13:31:38


Проверим данные на наличие дублей.

In [6]:
replenishment_of_accounts.duplicated().sum()

0

Рассмотрим датасет exchange_rates. Для обзора данных воспользуемся методами head() и info().

In [7]:
display(exchange_rates.head(10))
exchange_rates.info()

,rate_currency,rate_countercurrency,rate_value,rate_date
0,CNY,RUB,10.4217,2023-02-06
1,EUR,RUB,76.7344,2023-02-06
2,GEL,RUB,26.5863,2023-02-06
3,USD,RUB,70.3847,2023-02-06
4,CNY,RUB,10.3815,2023-02-07
5,EUR,RUB,76.0347,2023-02-07
6,GEL,RUB,26.7006,2023-02-07
7,USD,RUB,70.5991,2023-02-07
8,CNY,RUB,10.4331,2023-02-08
9,EUR,RUB,75.9087,2023-02-08


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   rate_currency         28 non-null     object 
 1   rate_countercurrency  28 non-null     object 
 2   rate_value            28 non-null     float64
 3   rate_date             28 non-null     object 
dtypes: float64(1), object(3)
memory usage: 1.0+ KB


Пропущенных значений нет. 

Изменим тип данных для поля rate_date на datetime.

In [8]:
exchange_rates['rate_date'] = pd.to_datetime(exchange_rates['rate_date'], format = '%Y-%m-%d')
exchange_rates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 4 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   rate_currency         28 non-null     object        
 1   rate_countercurrency  28 non-null     object        
 2   rate_value            28 non-null     float64       
 3   rate_date             28 non-null     datetime64[ns]
dtypes: datetime64[ns](1), float64(1), object(2)
memory usage: 1.0+ KB


Проверим данные на наличие дублей.

In [9]:
exchange_rates.duplicated().sum()

0

Определим какие валюты встречаются в полях rate_currency и rate_countercurrency

In [10]:
print('Поле rate_currency хранит следующие валюты:', exchange_rates['rate_currency'].unique())
print('Поле rate_countercurrency хранит следующие валюты:', exchange_rates['rate_countercurrency'].unique())

Поле rate_currency хранит следующие валюты: ['CNY' 'EUR' 'GEL' 'USD']
Поле rate_countercurrency хранит следующие валюты: ['RUB']


Вывод: 
Данные были проверены на наличие пропусков и дубликатов, были изменены типы данных там, где это было необходимо. Выявили, что в таблице учитывается перевод только из иностранной валюты в рубли. 

#### 1. Выведите максимальный размер пополнения в долларах за неделю, начинающуюся 6-го февраля.

Импортируем библиотеку для использования SQL.

In [11]:
from pandasql import sqldf

Пишем запрос.

In [12]:
query = """
select max(volume) as max_usd_replenishment
  from replenishment_of_accounts
 where date(operation_date) between '2023-02-06' and '2023-02-12'
   and currency = 'USD'
"""

result = sqldf(query, locals())

result.head(10)

,max_usd_replenishment
0,2648648.0


#### 2. Выведите средний размер пополнений за февраль понедельно, в разбивке по валютам. Результатом должна быть таблица формата «Год-номер недели, валюта, размер пополнения (с округлением до целого числа)».

In [13]:
query = """
select strftime('%Y',operation_date) as year
     , strftime('%W',operation_date) as week_number
     , currency
     , avg(volume) as avg_replenishment
  from replenishment_of_accounts
 where date(operation_date) between '2023-02-01' and '2023-02-28'
group by 1,2,3
order by week_number, currency
"""

result = sqldf(query, locals())

result.head(10)

,year,week_number,currency,avg_replenishment
0,2023,06,CNY,3.490972e+04
1,2023,06,EUR,7.611467e+03
2,2023,06,RUR,1.517762e+06
3,2023,06,USD,4.710190e+03


#### 3. Посчитайте долю клиентов, у которых были пополнения в нескольких валютах. Ответ округлите до десятых процента.

In [14]:
query = """

with usr as
(
select user_id
     , count(distinct currency) as diff_cur_cnt
  from replenishment_of_accounts
group by 1
)
select round(CAST(sum(case when diff_cur_cnt > 1 then 1 else 0 end) as real) / count(*), 3)
  from usr
;
"""

result = sqldf(query, locals())

result.head(10)

,"round(CAST(sum(case when diff_cur_cnt > 1 then 1 else 0 end) as real) / count(*), 3)"
0,0.003


#### 4. По каждому клиенту вывести дату первого пополнения, валюту, сумму в рублях (используйте вторую таблицу). Ответом выведите общую сумму в рублях по всем клиентам.

Выведем данные по каждому клиенту.

In [15]:
query = """
with rep as
(
select user_id
     , roa.volume * coalesce(er.rate_value, 1) as volume_rub
     , operation_date
     , currency
     , row_number() over (partition by user_id order by operation_date) rn -- находим по каждому пополнению клиента его порядковый номер
  from replenishment_of_accounts roa
left join exchange_rates er
    on roa.currency = er.rate_currency
   and roa.operation_date = er.rate_date
)
select user_id
     , volume_rub
     , operation_date
     , currency
  from rep
 where rn = 1
"""

result = sqldf(query, locals())

result.head(10)

,user_id,volume_rub,operation_date,currency
0,1403,296666.7,2023-02-07 17:46:13.000000,RUR
1,3311,195000.0,2023-02-08 11:44:09.000000,RUR
2,3594,2000000.0,2023-02-09 12:30:52.000000,RUR
3,3816,17140.0,2023-02-06 21:32:23.000000,CNY
4,3893,1500000.0,2023-02-07 12:00:34.000000,RUR
5,4963,500000.0,2023-02-10 17:55:08.000000,RUR
6,5643,7860.9,2023-02-08 17:34:13.000000,EUR
7,6622,1250000.0,2023-02-10 15:40:41.000000,RUR
8,6827,666.7,2023-02-06 15:31:13.000000,USD
9,7008,555559.5,2023-02-08 11:43:29.000000,RUR


Выведем общую сумму первых пополнений в рублях по всем клиентам.

In [16]:
query = """
with rep as
(
select user_id
     , roa.volume * coalesce(er.rate_value, 1) as volume_rub --в таблице курсов рублей нет, меняем null на 1 для рубля
     , operation_date
     , currency
     , row_number() over (partition by user_id order by operation_date) rn
  from replenishment_of_accounts roa
left join exchange_rates er
    on roa.currency = er.rate_currency
   and roa.operation_date = er.rate_date
)
select sum(volume_rub) as total_first_replenishment_rub
  from rep
 where rn = 1
"""

result = sqldf(query, locals())

result.head(10)

,total_first_replenishment_rub
0,2.230073e+10


#### 5. Посчитайте по каждому клиенту среднюю разницу в днях между его пополнениями, выведите медианное значение по всем клиентам.

In [17]:
query = """
with rep_dates as --найдем для каждого пополнения дату предыдущего пополнения этого пользователя
(
select user_id
     , date(operation_date) as operation_date
     , date(lag(operation_date) over (partition by user_id order by operation_date)) as prev_operation_date
  from replenishment_of_accounts roa
)
, rep_diff as --рассчитаем разницу в днях между соседними пополнениями
(
select user_id
     , julianday(operation_date) - julianday(prev_operation_date) as date_diff
  from rep_dates
 where prev_operation_date is not null --исключаем самое первое пополнение
)
, avg_diff as --средняя разница в днях между пополнениями каждого пользователя
(
select user_id
     , avg(date_diff) as avg_rep_day_by_user
  from rep_diff
group by 1
)
SELECT avg_rep_day_by_user as median_rep_day --функции медианы в sqlite нет, пришлось найти альтернативу в интернете
FROM avg_diff
ORDER BY 1
LIMIT 1
OFFSET (SELECT COUNT(*)
        FROM avg_diff) / 2
"""

result = sqldf(query, locals())

result.head(10)

,median_rep_day
0,1.0
